In [10]:
class lol(object):
    def __init__(self,a):
        self.a = a
    def reta(self):
        return self.a

class lmao():
    def __init__(self,lol):
        self.b = lol.a
    def retb(self):
        return self.b

In [16]:
c = lol(20)

#c.a = 23
c.reta()

20

In [17]:
d = lmao(c)
d.retb()

20

In [2]:
import numpy as np


In [4]:
np.load("../data/PMatrix_8cycle.npy")

array([[0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [9.60444897e-292, 4.67346072e-291, 1.75268447e-290, ...,
        8.94785185e-290, 2.38590772e-290, 4.90328909e-291],
       [8.10864208e-287, 3.92836556e-286, 1.46363621e-285, ...,
        8.75501142e-285, 2.34881375e-285, 4.84724344e-286],
       ...,
       [1.00421337e-009, 3.99663626e-009, 8.89959548e-009, ...,
        9.23389880e-009, 4.13300601e-009, 1.03627232e-009],
       [9.93385316e-010, 3.95415863e-009, 8.80706051e-009, ...,
        8.92361939e-009, 3.99060532e-009, 1.00002302e-009],
       [9.81007051e-010, 3.90577628e-009, 8.70235668e-009, ...,
        8.61392101e-009, 3.84859696e-009, 9.63897174e-010]])